In [3]:
# This notebook is for Peem's attempt to fit all models using labelled data.
# Last updated: 18th Feb 2022. 

from helper_fn import *

# Executive Summary

This Jupyter notebook is separated into these parts: 

1. Merging and cleaning multiple datasets. Each of the PEMAT labels are used to calculated labels on understandability, actionability, and medical information (already exists). 
2. Created a dictionary of PEMAT Question (Key) and its definition (Value).  
3. Performing EDA on 600 videos with PEMAT labels (see description in Part 2).

**NOTE:** I didn't remove entry 275 (URL = 'RQCq-FzeYgA') even though its PEMAT label is entirely NaN. Make sure to remove it in the final analysis. 

# Part 1: Cleaning and merging the dataset

In [13]:
# Downloading the labelled dataset.
df = pd.read_csv("content.csv", sep = ",")
df_label = pd.read_csv("label600.csv", sep = ",", encoding="ISO-8859-1")
df_label = df_label.T.set_index(0).T
df_label = df_label.dropna(axis = 1, how = "all")

# Downloading the separate metadata and engagement. 
df_meta = pd.read_csv("metadata.csv", sep = ",")
df_engagement = pd.read_csv("engagement.csv", sep = ",")

# Merge content, metadata, and engagement. Use the set operation to avoid column duplicates. 
first = set(df.columns)
second = set(df_meta.columns)
third = set(df_engagement.columns)
second = second - first
third = (third - second) - first
final = list(first) + list(second) + list(third)

# Concatenating all of the dataframe
dftemp = pd.concat([df, df_meta[second], df_engagement[third]], axis = 1)
df = dftemp

In [14]:
# TODO: Find how many topics there are for each video. 
df["relevantTopicIds"]

0                    ['/m/01k8wb', '/m/098wr', '/m/098wr']
1                                 ['/m/098wr', '/m/098wr']
2                                 ['/m/098wr', '/m/098wr']
3        ['/m/019_rr', '/m/0kt51', '/m/019_rr', '/m/0kt...
4                               ['/m/01k8wb', '/m/01k8wb']
                               ...                        
11138    ['/m/02wbm', '/m/019_rr', '/m/019_rr', '/m/02w...
11139    ['/m/02wbm', '/m/019_rr', '/m/098wr', '/m/098wr']
11140    ['/m/02wbm', '/m/019_rr', '/m/019_rr', '/m/02w...
11141                ['/m/019_rr', '/m/098wr', '/m/098wr']
11142                                         ['/m/098wr']
Name: relevantTopicIds, Length: 11143, dtype: object

In [15]:
# A function to clean the URL in the dataset. 
def get_id(full_link):
    temp = full_link.split("=")
    return temp[1]

# A function to obtain mapping between PEMAT criterion and its value.
def PEMAT_map(df):
    """
    This function accepts the dataframe and returns the dictionary that maps each PEMAT criteria 
    number to its description.
    """
    PEMAT_dict = {}
    for col_name in df.columns.tolist():
        try:
            split = col_name.split(".")
            key = split[0]
            temp = ""
            if len(split) > 1:
                # Concatenate every remaining string
                for i in range(1, len(split)):
                    temp += str(split[i])
            value = temp
            if key != 'This video contain high medical knowledge (0: low; 1: High)':
                PEMAT_dict[key] = value         
        except:
            pass
    PEMAT_dict["info"] = "This video contain high medical knowledge (0: low; 1: High)':"
    return PEMAT_dict

# Test if the dictionary is obtained.
PEMAT_dict = PEMAT_map(df_label)

def greater05(value):
    if value >= 0.5:
        return 1
    return 0

In [1]:
# Saving the PEMAT dictionary. This allows easier reference in case we need to use the raw data.
import pickle 
d_file = open("PEMAT_dict.pkl", "wb")
pickle.dump(PEMAT_dict, d_file)
d_file.close()

NameError: name 'PEMAT_dict' is not defined

In [18]:
d_file = open("PEMAT_dict.pkl", "rb")
output = pickle.load(d_file)

In [19]:
# Renaming the column
temp = {}
original_col = df_label.columns.tolist()
assert len(original_col) == len(list(PEMAT_dict.keys()))
for i, replacer in enumerate(list(PEMAT_dict.keys())):
    temp[original_col[i]] = replacer
df_label = df_label.rename(columns = temp)

# Obtaining the URL
df_label["URL"] = df_label.apply(lambda row: get_id(row["URL"]), axis = 1)

# Part 1.1: Checking the distribution of PEMAT labels/medical information

At this point, we have three objects — df_label (a PEMAT-labelled dataframe of 600 videos), df (a dataframe of 11,000 videos), and the PEMAT dictionary. I want to compute the labels and explore how many are classified as understandable, containing medical information, etc.

In [ ]:
actionable = [str(i) for i in [20,21,22,25]]
understandable = [str(i) for i in [1,3,4,5,8,9,10,11,13,14,18,19]]

# Change the type from string to int. 
df_label[actionable] = df_label[actionable].apply(lambda x: pd.to_numeric(x, errors='coerce'))
df_label[understandable] = df_label[understandable].apply(lambda x: pd.to_numeric(x, errors='coerce'))

# Calculate the mean
df_label["action"] = df_label[actionable].mean(axis = 1, skipna = True, numeric_only = True)
df_label["understand"] = df_label[understandable].mean(axis = 1, skipna = True, numeric_only = True)

# Apply indicator function whether the mean is greater than 0.5. 
# NOTE: There is a cleaner way of finding majority than applying lambda function.
# However, I chose this method in case we want to compute labels beyond using majority, 
# such as ignoring certain PEMAT criterion, weighting certain criteria more heavily, etc. 
df_label["action"] = df_label.apply(lambda row: greater05(row["action"]), axis = 1)
df_label["understand"] = df_label.apply(lambda row: greater05(row["understand"]), axis = 1)

In [21]:
# Checking if there's any column with only one value (i.e. if every element is the same, cannot be used.)
to_drop = []
for i, col in enumerate(df.columns):
    if df[col].value_counts().shape[0] == 1: # Only one type, so cannot be used for prediction. 
        to_drop.append(col)
df = df.drop(columns = to_drop)
print("Below are the list of variables I dropped")
for col in to_drop:
    print(col)

Below are the list of variables I dropped
has_tags
has_title
isCC
privacyStatus
uploadStatus
contentRating.ytRating
favoriteCount
audioTrackType
language


In [22]:
# After removing all columns with one input, I saved the 12k without labels into a dataframe.
# TODO: When Xiao provides the label for 12k, I will re-use it in the dataset.
df["categoryId"] = [str(category) for category in df["categoryId"]]
df.to_csv("merged_and_cleaned12k.csv")

In [30]:
df

,ARI,FleshReadingEase,Kincaid,active_verb,has_description,id,sentence_count,summary_words,transition_words,video_duration,...,channelVideoCount,keyword_decription_cosine,comment_total_words,negative_comment_count,comment_description_cosine,keyword_title_cosine,comment_title_cosine,postive_comment_count,comment_unique_words,neutral_comment_count
video_id,,,,,,,,,,,,,,,,,,,,,
bJKCEKCOeTw,17.31,-44.51,21.45,0.00,1.00,bJKCEKCOeTw,1.00,0.00,1.00,267.00,...,191.00,0.09,0.00,0.00,0.00,0.09,0.00,0.00,0.00,0.00
PBB4SHQHTbY,12.59,44.83,11.97,4.00,1.00,PBB4SHQHTbY,2.00,0.00,2.00,260.00,...,61.00,0.00,5.00,0.00,0.00,0.00,0.00,0.00,5.00,4.00
9zb0Oo0ryEI,14.32,28.18,13.64,4.00,1.00,9zb0Oo0ryEI,3.00,0.00,2.00,398.00,...,61.00,0.00,91.00,2.00,0.49,0.17,0.13,3.00,84.00,1.00
97uiV4RiSAY,19.30,13.90,16.04,1.00,1.00,97uiV4RiSAY,1.00,0.00,1.00,477.00,...,3.00,0.00,"1,346.00",14.00,1.27,0.20,0.83,66.00,"1,142.00",19.00
m-PozFAV1xE,21.88,-0.24,19.47,12.00,1.00,m-PozFAV1xE,7.00,0.00,2.00,870.00,...,39.00,0.03,63.00,0.00,0.03,0.00,0.00,7.00,59.00,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
LedWl2QTy40,18.83,32.11,16.94,99.00,1.00,LedWl2QTy40,20.00,0.00,6.00,119.00,...,715.00,0.02,15.00,0.00,0.09,0.00,0.22,2.00,14.00,0.00
GNghXeGlTdI,11.78,61.92,10.33,85.00,1.00,GNghXeGlTdI,23.00,0.00,8.00,684.00,...,322.00,0.04,20.00,0.00,0.28,0.00,0.27,1.00,18.00,0.00
d360OEFeVAQ,11.40,59.03,10.16,107.00,1.00,d360OEFeVAQ,32.00,0.00,7.00,167.00,...,494.00,0.01,21.00,0.00,0.22,0.00,0.00,1.00,18.00,1.00


# Part 1.3: Merging videos with PEMAT labels with overall 12k.
In this part, I merge df (12k) with df_label (600) based on URL. Afterwards, we will obtain a dataset with 600 labelled videos along with all of its information.


In [40]:
# Perform the matching based on 'URL' in df_label and 'id' in df.

tempdf = df_label[["URL","Duration","info", "action", "understand"]]

# Find all videos in the large dataset with labels. 
# Results: The dataset has some duplicate. 
tempdf = tempdf.set_index("URL")
try:
    df = df.set_index("video_id")
except:
    pass
newdf = tempdf.join(df)

# Dropping duplicate entries and storing the cleaned dataset
try:
    newdf.reset_index(inplace = True)
except:
    pass 
newdf = newdf.rename(columns = {"index":"URL"})
newdf = newdf.drop_duplicates("URL")
df600 = newdf

In [42]:
# Convert every features whose values can be interpreted as numbers.
for col in df600.columns.tolist():
    df600[col] = pd.to_numeric(df600[col], errors = "ignore")

# Part 1.4: Use Xiao's readability.
TODO

In [ ]:
# Download the dataframe on readability for each video. Rename the dataframe
readability_df = readability_df.set_index("video_id")
df = df.set_index("URL")
# Re-assign readability indices
for index in df.index.tolist():
    if index in readability_df.index.tolist():
        df.loc[index, "ARI"] = readability_df.loc[index, "ari"] 
        df.loc[index, "FleshReadingEase"] = readability_df.loc[index, "flesch"]
        df.loc[index, "Kincaid"] = readability_df.loc[index, "kincaid"] 
# Save all files
df_label.to_csv("rawPEMAT.csv")
df.to_csv("merged_and_cleaned600.csv")

In [ ]:
df["categoryId"] = [str(category) for category in df["categoryId"]]
df.to_csv("merged_and_cleaned600.csv")

# Part 2: Checking PEMAT input per questions

In the above part, I have merged Xiao's datasets and created two versions: merged_and_cleaned with labels (600) and merged_and_cleaned without labels (11000). The remaining task would be to join them by row_id, if need be. However, it's interesting to examine df_label itself to see what the distribution of each questions are.

## Missing values
For most of the columns, there are only few missing entries. The noteworthy columns with high misses are 13, 18, 19, 25, which corresponds to clarity of simple graphs/illustrations/etc. These questions may not be applicable to all diabetes videos. Therefore, there's nothing egregious about missingness. 

## Strange values
Medical information (i.e. 'info') has a lot of non-sense values. 

## Observations
1. 75% of the videos are understandable (1-19), 25% are not. A simple rule of outputting 1 (i.e. every video is understandable) will achieve 75% accuracy, so **need to think about alternative metrics**. 47% of videos are actionable (corresponding to Question 20,21,22,25).

2. Because some PEMAT questions have NaN, I cannot calculate the correlation between each of the response questions.
TODO: Ask Larry/Nynke if there's a need to calculate the correlation.

3. Actionability and understandability have the correlation coefficient of 0.128. This value is very low. 

4. Duration (unit: second) is skewed heavily to the right — there are some very lengthy videos. **Need to log-scale if included in the final model.**

5. PEMAT criteria with mostly zero entries are 8,9,19,22 (check PEMAT_dict). They all correspond to not breaking down information into small-chunks/actionable steps or lack of informative headers. **This could inform why videos we classify as zero are not understandable/actionable**. 

6. "11" (summary) and "25" (graphs/charts to take actions) do not receive good scores. 

In [ ]:
# Checking missing values of PEMAT response in each column. 
df_label.isna().sum()

In [ ]:
# Distribution
df_label["Duration"] = pd.to_numeric(df_label["Duration"],)
numerical_col = df_label.columns.tolist()
numerical_col.remove("Title")
numerical_col.remove("URL")
fare = ["min", "max", "median", "mean"]
temp = {}
for col in numerical_col:
    temp[col] = fare
df_label.agg(temp)

In [ ]:
# A code to find the covaraince matrix for each of the PEMAT questions. 
temp = []
for col in  df_label.columns.tolist()[3:19]:
    temp.append(df_label[col].tolist())
temp = np.array(temp)

# TODO: uncomment. np.corrcoef(temp)

# Find the correlation between understand and action
act_under = np.array([df_label["action"].tolist(), df_label["understand"]])
np.corrcoef(act_under)

In [ ]:
# Finding the location where info label is unavailable.
np.where(df_label["info"].isna() == True)
df = df.drop(index = [275])